In [ ]:
# import fl_score error

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score
from sklearn.metrics import roc_curve, auc

from sklearn.model_selection import learning_curve, validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [6]:
# data

In [ ]:
df = pd.read_csv('data.csv')
df.info()
df['diagnosis'].unique() # M = malignant, B = benign

def func1(x):
    if 'M' in x:
        return 1
    else:
        return 0

df['평점'] = df['diagnosis'].apply(func1)
df.head()
